In [ ]:
!wget https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/examples/training/matryoshka/matryoshka_sts.py

In [ ]:
!pip install sentence-transformers
!pip install optimum[onnxruntime] transformers

In [ ]:
!python matryoshka_sts.py intfloat/multilingual-e5-large

In [4]:
!ls -lh

total 16K
drwxr-xr-x 2 root root 4.0K Mar  7 14:25 datasets
-rw-r--r-- 1 root root 4.7K Mar  7 14:24 matryoshka_sts.py
drwxr-xr-x 3 root root 4.0K Mar  7 14:29 output


In [7]:
!ls -lh output

total 4.0K
drwxr-xr-x 4 root root 4.0K Mar  7 14:39 matryoshka_sts_intfloat-multilingual-e5-large-2024-03-07_14-25-25


In [8]:
!ls -lh ./output/matryoshka_sts_intfloat-multilingual-e5-large-2024-03-07_14-25-25

total 2.2G
drwxr-xr-x 2 root root 4.0K Mar  7 14:29 1_Pooling
-rw-r--r-- 1 root root 4.0K Mar  7 14:35 README.md
-rw-r--r-- 1 root root  716 Mar  7 14:35 config.json
-rw-r--r-- 1 root root  165 Mar  7 14:35 config_sentence_transformers.json
drwxr-xr-x 2 root root 4.0K Mar  7 14:29 eval
-rw-r--r-- 1 root root 2.1G Mar  7 14:35 model.safetensors
-rw-r--r-- 1 root root  229 Mar  7 14:35 modules.json
-rw-r--r-- 1 root root   53 Mar  7 14:35 sentence_bert_config.json
-rw-r--r-- 1 root root 4.9M Mar  7 14:35 sentencepiece.bpe.model
-rw-r--r-- 1 root root  301 Mar  7 14:39 similarity_evaluation_sts-test_results.csv
-rw-r--r-- 1 root root  964 Mar  7 14:35 special_tokens_map.json
-rw-r--r-- 1 root root  17M Mar  7 14:35 tokenizer.json
-rw-r--r-- 1 root root 1.2K Mar  7 14:35 tokenizer_config.json


In [ ]:
from optimum.onnxruntime import ORTModelForFeatureExtraction
import onnxruntime as onnxrt
from transformers import AutoTokenizer, pipeline

In [ ]:
save_directory = "/tmp/matryoshka-sts-intfloat-multilingual-e5-large-onnx"

original_model = ORTModelForFeatureExtraction.from_pretrained(
    "./output/matryoshka_sts_intfloat-multilingual-e5-large-2024-03-07_14-25-25",
    export=True
)

original_tokenizer = AutoTokenizer.from_pretrained(
    "./output/matryoshka_sts_intfloat-multilingual-e5-large-2024-03-07_14-25-25"
)

original_model.save_pretrained(save_directory)
original_tokenizer.save_pretrained(save_directory)

In [13]:
!ls -lh /tmp/matryoshka-sts-intfloat-multilingual-e5-large-onnx

total 2.2G
-rw-r--r-- 1 root root  772 Mar  7 14:47 config.json
-rw-r--r-- 1 root root 534K Mar  7 14:47 model.onnx
-rw-r--r-- 1 root root 2.1G Mar  7 14:47 model.onnx_data
-rw-r--r-- 1 root root 4.9M Mar  7 14:47 sentencepiece.bpe.model
-rw-r--r-- 1 root root  964 Mar  7 14:47 special_tokens_map.json
-rw-r--r-- 1 root root  17M Mar  7 14:47 tokenizer.json
-rw-r--r-- 1 root root 1.4K Mar  7 14:47 tokenizer_config.json


In [14]:
import multiprocessing

from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer, pipeline
import onnxruntime as onnxrt

onnxrt_options = onnxrt.SessionOptions()

onnxrt_options.execution_mode = onnxrt.ExecutionMode.ORT_SEQUENTIAL
onnxrt_options.intra_op_num_threads = multiprocessing.cpu_count()

onnxrt_options.graph_optimization_level = onnxrt.GraphOptimizationLevel.ORT_ENABLE_ALL
onnxrt_options.add_session_config_entry('session.intra_op.allow_spinning', '1')

model = ORTModelForFeatureExtraction.from_pretrained(
    "/tmp/matryoshka-sts-intfloat-multilingual-e5-large-onnx",
    session_options=onnxrt_options,
    providers=['CPUExecutionProvider']
)

tokenizer = AutoTokenizer.from_pretrained("/tmp/matryoshka-sts-intfloat-multilingual-e5-large-onnx")

onnx_extractor = pipeline("feature-extraction", model=model, tokenizer=tokenizer)

In [16]:
text = "My name is Philipp and I live in Germany."
embedding = onnx_extractor(text)

In [19]:
print(len(embedding[0][0]))
print(embedding[0][0][:5])

1024
[-0.6574172973632812, -0.7142754197120667, -0.02111353725194931, -1.528099775314331, 1.1149659156799316]


In [22]:
text = "This is an experiment."
embedding = onnx_extractor(text)

In [23]:
print(len(embedding[0][0]))
print(embedding[0][0][:5])

1024
[0.5266503095626831, -1.2940523624420166, -0.2779468595981598, -2.410278797149658, 0.6889562010765076]


In [33]:
# Requested matryoshka dimensions
dimensions = 256

matryoshka_embedding = embedding[0][0][:dimensions]

print(len(matryoshka_embedding))

256
